In [1]:
# Imports
import warnings
import os
import sys

import numpy as np
import numpy.ma as ma
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.lines as mlines

import geopandas as gpd
from geopandas import GeoDataFrame as gdf
from geopandas import GeoSeries as gs
from shapely.geometry import Point, Polygon

import contextily as ctx
import earthpy as et
import earthpy.plot as ep

In [2]:
# Check path and set working directory.
wd_path = os.path.join(et.io.HOME, 'earth-analytics', 'data')
if os.path.exists(wd_path):
    os.chdir(wd_path)
else:
    print("Path does not exist")

In [3]:
# Download Data stored on figshare
# Wdpa polygon files
et.data.get_data(url="https://ndownloader.figshare.com/files/22847855")

# Future dams
et.data.get_data(url="https://ndownloader.figshare.com/files/22486157")

'/home/jovyan/earth-analytics/data/earthpy-downloads/future_dams_2015.csv'

In [4]:
# define the path to files
wdpa_points_path = os.path.join(wd_path,
    "earthpy-downloads", "WDPA_Apr2020-shapefile-points", "WDPA_Apr2020-shapefile-points.shp")

future_dams_path = os.path.join(wd_path, "earthpy-downloads", "future_dams_2015.csv")

In [5]:
wdpa_points = gpd.read_file(wdpa_points_path)

wdpa_points



,WDPAID,WDPA_PID,PA_DEF,NAME,ORIG_NAME,DESIG,DESIG_ENG,DESIG_TYPE,IUCN_CAT,INT_CRIT,...,GOV_TYPE,OWN_TYPE,MANG_AUTH,MANG_PLAN,VERIF,METADATAID,SUB_LOC,PARENT_ISO,ISO3,geometry
0,4195.0,4195,1,Queen Elizabeth,Queen Elizabeth,National Park,National Park,National,II,Not Applicable,...,Federal or national ministry or agency,Not Reported,Not Reported,Not Reported,Not Reported,983,Not Reported,SLB,SLB,MULTIPOINT (160.20000 -9.53330)
1,4362.0,4362,1,Kolombangara Forest Reserve,Kolombangara Forest Reserve,Controlled Forest,Controlled Forest,National,Not Assigned,Not Applicable,...,Local communities,Communal,KIBCA Board,Draft,State Verified,1932,Not Reported,SLB,SLB,MULTIPOINT (157.08330 -8.00000)
2,316912.0,316912,1,Naro - Terrestrial,Naro - Terrestrial,Controlled Forest,Controlled Forest,Not Applicable,Not Assigned,Not Applicable,...,Local communities,Communal,Not Reported,Not Reported,State Verified,1932,Not Reported,SLB,SLB,MULTIPOINT (160.20000 -9.53330)
3,316913.0,316913,1,Komarindi,Komarindi,Conservation Area,Conservation Area,National,V,Not Applicable,...,Not Reported,Not Reported,Not Reported,Not Reported,Not Reported,983,Not Reported,SLB,SLB,MULTIPOINT (160.00000 -9.00000)
4,316914.0,316914,1,Langa Langa Lagoon,Langa Langa Lagoon,Reserve,Reserve,National,Not Reported,Not Applicable,...,Not Reported,Not Reported,Not Reported,Not Reported,Not Reported,983,Not Reported,SLB,SLB,MULTIPOINT (161.00000 -9.00000)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21234,207283.0,207283,1,"Name Unknown, site No. 598 (Sverdlovskaya obl.)","Name Unknown, site No. 598 (Sverdlovskaya obl.)",Not Reported,Not Reported,National,Not Reported,Not Applicable,...,Not Reported,Not Reported,Not Reported,Not Reported,State Verified,840,RU-SVE,RUS,RUS,MULTIPOINT (61.28414 59.00340)
21235,209456.0,209456,1,"Name Unknown, site No. 999 (Yaroslavskaya obl.)","Name Unknown, site No. 999 (Yaroslavskaya obl.)",Nature Monument,Nature Monument,National,III,Not Applicable,...,Not Reported,Not Reported,Not Reported,Not Reported,State Verified,840,RU-YAR,RUS,RUS,MULTIPOINT (39.29639 56.65453)
21236,209499.0,209499,1,"Name Unknown, site No. 41 (Krasnoyarskiy kray)","Name Unknown, site No. 41 (Krasnoyarskiy kray)",Not Reported,Not Reported,National,Not Reported,Not Applicable,...,Not Reported,Not Reported,Not Reported,Not Reported,State Verified,840,RU-KYA,RUS,RUS,MULTIPOINT (95.75592 57.03109)
21237,209500.0,209500,1,"Name Unknown, site No. 59 (Krasnoyarskiy kray)","Name Unknown, site No. 59 (Krasnoyarskiy kray)",Not Reported,Not Reported,National,Not Reported,Not Applicable,...,Not Reported,Not Reported,Not Reported,Not Reported,State Verified,840,RU-KYA,RUS,RUS,MULTIPOINT (94.96582 56.18639)


In [6]:
print(wdpa_points.crs)

epsg:4326


In [13]:
wdpa_points = wdpa_points.to_crs('epsg:4087')

In [7]:
# Open the dams csv files with pandas
dams_source_path = os.path.join("earthpy-downloads", "future_dams_2015.csv")
dams_df = pd.read_csv(dams_source_path)

In [8]:
dams_df

,DAM_ID,Project name,Continent,Country,Main_river,Major Basin,Capacity (MW),LAT_cleaned,Lon_Cleaned,Stage,Start,End,Reference 1,Reference 2,Reference 3
0,0,Susitna-Watana,North America,United States,Susitna,Pacific and Arctic Coast 1,600.00,62.786772,-148.025780,P,2012.0,2023.0,http://www.hydroreform.org/projects/susitna-wa...,NaN,NaN
1,1,Site C,North America,Canada,Peace,Mackenzie,900.00,56.225245,-120.950245,P,2014.0,2020.0,http://en.wikipedia.org/wiki/Site_C_dam,https://www.sitecproject.com/sites/default/fil...,NaN
2,2,Gull Island,North America,Canada,Churchill,Churchill,2250.00,53.032879,-61.212954,U,NaN,2028.0,http://en.wikipedia.org/wiki/Lower_Churchill_P...,http://www.nalcorenergy.com/lower-churchill-pr...,NaN
3,3,Muskrat,North America,Canada,Churchill,Churchill,824.00,53.244625,-60.773792,U,NaN,2017.0,http://en.wikipedia.org/wiki/Lower_Churchill_P...,http://www.nalcorenergy.com/lower-churchill-pr...,NaN
4,4,Conawapa,North America,Canada,Nelson,Saskatchewan - Nelson,1485.00,56.535417,-94.029465,P,NaN,2022.0,http://www.energymanitoba.org/hp_conawapa.htm,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3695,3777,Esperanza De Atirro,North America,Costa Rica,Parismina,Southern Central America,3.40,9.810665,-83.648165,P,NaN,NaN,Plantas y proyectos,NaN,NaN
3696,3778,Rio Banano,North America,Costa Rica,Estrella,Southern Central America,20.00,9.892825,-83.164583,P,NaN,NaN,Plantas y proyectos,NaN,NaN
3697,3779,Tarcoles,North America,Costa Rica,Tárcoles / Virilla / Colorado,Southern Central America,19.27,9.916664,-84.466669,P,NaN,NaN,Plantas y proyectos,NaN,NaN
3698,3780,Ruf,North America,Costa Rica,Tárcoles / Virilla / Colorado,Southern Central America,20.00,9.910417,-84.487152,P,NaN,NaN,Plantas y proyectos,NaN,NaN


In [9]:
# Covert the pandas dataframe to a shapefile for plotting
# Set output path for shp
dams_path = os.path.join('fhred-proposed-dams')
if not os.path.exists(dams_path):
    os.mkdir(dams_path)

note about crs
World Equidistant Cylindrical, datum WGS84, units meters, EPSG 4087


In [10]:
# Define the geometry for the points
geometry = [Point(xy) for xy in zip(dams_df.Lon_Cleaned, dams_df.LAT_cleaned)]
crs = {'init': 'epsg:4087'}
geo_df = gdf(dams_df, crs=crs, geometry=geometry)
geo_df.to_file(driver='ESRI Shapefile', filename=os.path.join(
    dams_path, 'proposed_dams.shp'))

/opt/conda/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [11]:
# Open the proposed dams shapefile with geopandas
proposed_dams = gpd.read_file(os.path.join(dams_path, "proposed_dams.shp"))

proposed_dams

,DAM_ID,Project na,Continent,Country,Main_river,Major Basi,Capacity (,LAT_cleane,Lon_Cleane,Stage,Start,End,Reference,Referenc_1,Referenc_2,geometry
0,0,Susitna-Watana,North America,United States,Susitna,Pacific and Arctic Coast 1,600.00,62.786772,-148.025780,P,2012.0,2023.0,http://www.hydroreform.org/projects/susitna-wa...,None,None,POINT (-148.02578 62.78677)
1,1,Site C,North America,Canada,Peace,Mackenzie,900.00,56.225245,-120.950245,P,2014.0,2020.0,http://en.wikipedia.org/wiki/Site_C_dam,https://www.sitecproject.com/sites/default/fil...,None,POINT (-120.95024 56.22524)
2,2,Gull Island,North America,Canada,Churchill,Churchill,2250.00,53.032879,-61.212954,U,NaN,2028.0,http://en.wikipedia.org/wiki/Lower_Churchill_P...,http://www.nalcorenergy.com/lower-churchill-pr...,None,POINT (-61.21295 53.03288)
3,3,Muskrat,North America,Canada,Churchill,Churchill,824.00,53.244625,-60.773792,U,NaN,2017.0,http://en.wikipedia.org/wiki/Lower_Churchill_P...,http://www.nalcorenergy.com/lower-churchill-pr...,None,POINT (-60.77379 53.24463)
4,4,Conawapa,North America,Canada,Nelson,Saskatchewan - Nelson,1485.00,56.535417,-94.029465,P,NaN,2022.0,http://www.energymanitoba.org/hp_conawapa.htm,None,None,POINT (-94.02947 56.53542)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3695,3777,Esperanza De Atirro,North America,Costa Rica,Parismina,Southern Central America,3.40,9.810665,-83.648165,P,NaN,NaN,Plantas y proyectos,None,None,POINT (-83.64817 9.81067)
3696,3778,Rio Banano,North America,Costa Rica,Estrella,Southern Central America,20.00,9.892825,-83.164583,P,NaN,NaN,Plantas y proyectos,None,None,POINT (-83.16458 9.89282)
3697,3779,Tarcoles,North America,Costa Rica,Tárcoles / Virilla / Colorado,Southern Central America,19.27,9.916664,-84.466669,P,NaN,NaN,Plantas y proyectos,None,None,POINT (-84.46667 9.91666)
3698,3780,Ruf,North America,Costa Rica,Tárcoles / Virilla / Colorado,Southern Central America,20.00,9.910417,-84.487152,P,NaN,NaN,Plantas y proyectos,None,None,POINT (-84.48715 9.91042)


In [14]:
# 50 km buffer
proposed_dams['geometry'] = proposed_dams.buffer(50 * 1000)

data_intersect_50 = gpd.overlay(wdpa_points, proposed_dams, how='intersection')
wdpa_buffer_50 = ((data_intersect_50.area.sum())/1000000).round(1)

In [15]:
wdpa_buffer_50

0.0